# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [6]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [7]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
import collections
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.
3. Turn the labels into class vectors

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [10]:
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenized data
    """
    # TODO: Implement
    MAX_NB_WORDS = len(english_words_counter)
    
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    return sequences, tokenizer

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sample {}'.format(sample_i + 1))
    print('  {}'.format(sent))
    print('  {}'.format(token_sent))

{'sentence': 4, 'over': 5, 'is': 18, 'lazy': 6, 'the': 3, 'won': 11, 'of': 9, 'a': 2, 'dog': 10, 'lexicography': 17, 'study': 8, 'short': 12, 'by': 13, 'prize': 14, 'jumps': 7, 'quick': 1, 'my': 16, 'this': 15, 'brown': 19, 'jove': 20, 'fox': 21}

Sample 1
  The quick brown fox jumps over the lazy dog .
  [3, 1, 19, 21, 7, 5, 3, 6, 10]
Sample 2
  By Jove , my quick study of lexicography won a prize .
  [13, 20, 16, 1, 8, 9, 17, 11, 2, 14]
Sample 3
  This is a short sentence .
  [15, 18, 2, 12, 4]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [11]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad data
    """
    # TODO: Implement
    x_pad = pad_sequences(x, maxlen=length, dtype='int32')
    return x_pad

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sample {}'.format(sample_i + 1))
    print('  {:>2} Length - {}'.format(len(token_sent), np.array(token_sent)))
    print('  {:>2} Length - {}'.format(len(pad_sent), pad_sent))

Sample 1
   9 Length - [ 3  1 19 21  7  5  3  6 10]
  10 Length - [ 0  3  1 19 21  7  5  3  6 10]
Sample 2
  10 Length - [13 20 16  1  8  9 17 11  2 14]
  10 Length - [13 20 16  1  8  9 17 11  2 14]
Sample 3
   5 Length - [15 18  2 12  4]
  10 Length - [ 0  0  0  0  0 15 18  2 12  4]


### Categorical (IMPLEMENTATION)
Since we're predicting on categorical data, we should turn the classes into class vectors using Keras's [`to_categorical`](https://keras.io/utils/#to_categorical) function.  The shape returned by `to_categorical` will lose the sequence length, so You'll have to add back the sequence length using NumPy's `reshape`.

In [12]:
from keras.utils import to_categorical


def categorize(x, n_categories):
    """
    Turn data into binary class matrices
    """
    # TODO: Implement
    x_shape = x.shape
    return to_categorical(x, n_categories).reshape((*x_shape, n_categories))

# Categorize Padded output
test_categories = categorize(test_pad, len(text_tokenizer.word_index) + 1)
for sample_i, (pad_sent, test_categorie) in enumerate(zip(test_pad, test_categories)):
    print('Sample {}'.format(sample_i + 1))
    print(pad_sent)
    print(test_categorie)

Sample 1
[ 0  3  1 19 21  7  5  3  6 10]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]]
Sample 2
[13 20 16  1  8  9 

### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [ ]:
def preprocess(x, y):
    pad_length = None
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x, pad_length)
    preprocess_y = pad(preprocess_y, pad_length)

    preprocess_y = categorize(preprocess_y, len(y_tk.word_index) + 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Encoder-Decoder
- Model 4 is a Bidirectional RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.

Each model will want the input in a different form.  While you did preprocess the English sentences, you'll have to add additional padding and/or reshape it to 3 dimensions for the right situation.
### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [ ]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam


def model_one(x, y, x_tk, y_tk):
    # TODO: Implement
    pass


model_one(*preprocess(english_sentences, french_sentences))

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [ ]:
from keras.layers.embeddings import Embedding


def model_two(x, y, x_tk, y_tk):
    # TODO: Implement
    pass


model_two(*preprocess(english_sentences, french_sentences))

### Model 3: Encoder-Decoder (IMPLEMENTATION)
![RNN](images/encoder_decoder.png)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
from keras.layers import RepeatVector


def model_three(x, y, x_tk, y_tk):
    # TODO: Implement
    pass


model_three(*preprocess(english_sentences, french_sentences))

### Model 4: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [ ]:
from keras.layers import Bidirectional


def model_four(x, y, x_tk, y_tk):
    # TODO: Implement
    pass


model_four(*preprocess(english_sentences, french_sentences))

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous 4 models to create a model that incorporates embedding, encoder-decoder, and bidirectional rnn into one model.

In [ ]:
def model_final(x, y, x_tk, y_tk):
    # TODO: Implement
    return None

## Prediction

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    model = model_final(x, y, x_tk, y_tk)

    # Set the tokenizer for the English data
    x_tokenizer = x_tk
    # Set the tokenizer for the French data
    y_tokenizer = y_tk
    y_id_to_word = {value: key for key, value in y_tokenizer.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck .'
    sentence = [x_tokenizer.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune.')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(*preprocess(english_sentences, french_sentences))

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.